## 13-9. 프로젝트 - Movielens 영화 추천 실습
이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.

별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.

별점을 시청횟수로 해석해서 생각하겠습니다.

또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
Cloud Storage에 미리 업로드된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결해 줍니다.

Cloud shell에서 아래 명령어를 입력해 주세요.

$ mkdir -p ~/aiffel/recommendata_iu/data/ml-1m

$ ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m


In [1]:
import numpy as np
import scipy
import implicit
import pandas as pd
import os,sys,copy,time

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


#### 1) 데이터 준비와 전처리

Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.



In [2]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
print(orginal_data_size, ratings.head())

1000209    user_id  movie_id  ratings  timestamp
0        1      1193        5  978300760
1        1       661        3  978302109
2        1       914        3  978301968
3        1      3408        4  978300275
4        1      2355        5  978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
print(movies.shape, movies.head())

(3883, 3)    movie_id                               title                         genre
0         1                    Toy Story (1995)   Animation|Children's|Comedy
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy
2         3             Grumpier Old Men (1995)                Comedy|Romance
3         4            Waiting to Exhale (1995)                  Comedy|Drama
4         5  Father of the Bride Part II (1995)                        Comedy


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

In [7]:
#[[ your code ]]
## 영화title명 확인위한 병합파일 merge별도생성(ratings  + movies)하여 보조자료로 활용
# 병합전 결측치여부 : 결측치 없슴
print("병합전 결측치",ratings.isnull().sum(), movies.isnull().sum())

# 병합 및 merge생성
merge = pd.merge(ratings,movies, how='outer',on='movie_id')

# 병합후 결측치여부 : 결측치 있슴, ratings에는 없는 영화 255개 존재
print("병합후 merge 결측치",merge.isnull().sum())
print("병합후 merge ",merge[-255:])

# ratings에 없는 영화 별도저장: 필요시 사용
movies_notin_rating = merge[-255:]

# 별도저장후 merge의 결측치 drop 처리
merge = merge[:-255]

# 결측치제거후 merge.head(), tail()로 data 확인
print(merge.head(3), merge.tail(3))
print("merge.shape",merge.shape)

병합전 결측치 user_id      0
movie_id     0
counts       0
timestamp    0
dtype: int64 movie_id    0
title       0
genre       0
dtype: int64
병합후 merge 결측치 user_id      255
movie_id       0
counts       255
timestamp    255
title          0
genre          0
dtype: int64
병합후 merge          user_id  movie_id  counts  timestamp  \
836478      NaN        51     NaN        NaN   
836479      NaN       109     NaN        NaN   
836480      NaN       115     NaN        NaN   
836481      NaN       127     NaN        NaN   
836482      NaN       133     NaN        NaN   
...         ...       ...     ...        ...   
836728      NaN      3829     NaN        NaN   
836729      NaN      3856     NaN        NaN   
836730      NaN      3891     NaN        NaN   
836731      NaN      3904     NaN        NaN   
836732      NaN      3907     NaN        NaN   

                                                    title  \
836478                              Guardian Angel (1994)   
836479                Hea

#### 2) 분석해 봅시다.
###### ratings에 있는 유니크한 영화 개수: 3628개 (별점3점이상 전체데이터수 836478개중)

###### ratings에 있는 유니크한 사용자 수: 6039 개(별점3점이상 전체데이터수 836478개중)

###### 가장 인기 있는 영화 30개(인기순): 총30개중 상위 5개만 예시 (병합파일 merge활용하여 영화제목 예시)
2858      American Beauty (1999)                                 Comedy|Drama                           3428

260       Star Wars: Episode IV - A New Hope (1977)              Action|Adventure|Fantasy|Sci-Fi        2991

1196      Star Wars: Episode V - The Empire Strikes Back (1980)  Action|Adventure|Drama|Sci-Fi|War      2990

1210      Star Wars: Episode VI - Return of the Jedi (1983)      Action|Adventure|Romance|Sci-Fi|War    2883

480       Jurassic Park (1993)             

###### 첫 번째 유저가 어떤 영화를 보는지 확인: 총53개중 상위 5개만 예시
0                 One Flew Over the Cuckoo's Nest (1975)

1725                    James and the Giant Peach (1996) 

2250                                 My Fair Lady (1964) 

2886                              Erin Brockovich (2000)  

4201                                Bug's Life, A (1998)   

5904                          Princess Bride, The (1987) 

###### 메타 데이터 movies의 영화제목 총수:  3883개 


In [8]:
#[[ your code ]]
# ratings에 있는 유니크한 영화 개수: 3628개, 전체데이터수 836478개
print("unique_movie_num", ratings['movie_id'].nunique(), "전체 데이터개수",ratings.shape[0])

#ratings에 있는 유니크한 사용자 수: 6039개 전체데이터수 836478개
print("unique_user_num", ratings['user_id'].nunique(), "전체 데이터개수",ratings.shape[0])

# 가장 인기 있는 영화 30개(인기순): movie_id, 영화제목, 장르, counts 4항목 출력
movie_count = merge.groupby(['movie_id','title','genre'])['user_id'].count()
print("가장 인기 있는 영화 30선(movie_id, 영화제목, 장르, counts): ",\
 movie_count.sort_values(ascending=False).head(30)) #[['user_id','movie_id','counts','title']])

# 유저별 몇 편의 영화를 보았는지에 대한 통계: 1인평균 138편,표준편차 156편, median 81편
user_count = merge.groupby('user_id')['movie_id'].count()
print(user_count.describe())

# 첫 번째 유저가 어떤 영화를 보는지 확인
condition = (merge['user_id']== merge.loc[0, 'user_id'])
firid_movie = merge.loc[condition]
print("첫 번째 유저가 어떤 영화를 보는지 확인",len(firid_movie),firid_movie)

unique_movie_num 3628 전체 데이터개수 836478
unique_user_num 6039 전체 데이터개수 836478
가장 인기 있는 영화 30선(movie_id, 영화제목, 장르, counts):  movie_id  title                                                  genre                              
2858      American Beauty (1999)                                 Comedy|Drama                           3211
260       Star Wars: Episode IV - A New Hope (1977)              Action|Adventure|Fantasy|Sci-Fi        2910
1196      Star Wars: Episode V - The Empire Strikes Back (1980)  Action|Adventure|Drama|Sci-Fi|War      2885
1210      Star Wars: Episode VI - Return of the Jedi (1983)      Action|Adventure|Romance|Sci-Fi|War    2716
2028      Saving Private Ryan (1998)                             Action|Drama|War                       2561
589       Terminator 2: Judgment Day (1991)                      Action|Sci-Fi|Thriller                 2509
593       Silence of the Lambs, The (1991)                       Drama|Thriller                         2498
1198      Raide

#### 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [9]:
#[[ your code ]]
# 본인이 좋아하는 영화제목 5개 추가 : 약식 명칭임
my_favorite_title = ['Legends of the Fall' , 'Jurassic Park' ,'Braveheart' ,'Terminator' ,'Dances with Wolves']

# my_favorit_title 를 movie_id로 변경
my_favorite = []
my_mov_real_title = []
for mft in my_favorite_title:
    print(mft)
    mov_id = merge[merge['title'].apply(lambda x: x.find(mft)!=-1)]['movie_id'].values[0]
    mov_real_title = merge.loc[merge['movie_id']==mov_id, 'title'].values[0]  #apply(lambda x: x.find(mft)!=-1).values[0]
    print("mov_id",mov_id, "mov_real_title",mov_real_title)
    my_favorite.append(mov_id)
    my_mov_real_title.append(mov_real_title)
# my_favorite, movie_id, real_title 확인
print("my_favorite_movie_id 확인: ", my_favorite)
print("my_favorite_movie_real_title 확인: ", my_mov_real_title)

# 본인이 좋아하는 영화제목 5개 추가 : 정식명칭임()
# 정식명칭: ['Legends of the Fall (1994)', 'Lost World: Jurassic Park, The (1997)', 'Braveheart (1995)', 'Terminator 2: Judgment Day (1991)', 'Dances with Wolves (1990)']
my_favorite_title = my_mov_real_title 

# 'bluesky'이라는 user_id가 위 영화를 5회씩 보았다고 가정
my_watchlist = pd.DataFrame({'user_id': ['bluesky']*5, 'movie_id': my_favorite, 'counts':[5]*5})

# ratings에 추가
if not ratings.isin({'user_id':['bluesky']})['user_id'].any():  
    ratings = ratings.append(my_watchlist)                          
print("ratings.tail(5)",ratings.tail(5))       # 잘 추가되었는지 확인해 봅시다.))

# 영화제목확인용 보조자료 merge에도 추가
if not merge.isin({'user_id':['bluesky']})['user_id'].any():  
    merge = merge.append(my_watchlist) 
print("merge.tail(5)",merge.tail(5))       # 잘 추가되었는지 확인해 봅시다.      ))


## ratings에 추가된 my favorite 추가해서, 재통합 id 산출
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movies_unique = ratings['movie_id'].unique()

# 유저, 영화 indexing 하는 코드: Original 유저id,Original 영화id가 enumerate의 순서로 인덱스가 바뀜
# ==> 그런데, 영화의 제목명을 제대로 파악하려면, merge 파일의 title 컴럼과 movie_id을 이용해서 찾아야
# ==> 정확한 영화제목을 찾을수 있음을 주의해야함 
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movies_to_idx = {v:k for k,v in enumerate(movies_unique)}   # 신규로 부여된 인덱스임,이 인덱스를 사용하되,
# ==> merge 파일의 title 컴럼과 movie_id을 이용해서 찾을 것

# 인덱싱이 잘 되었는지 확인
print(user_to_idx['bluesky'], len(user_to_idx))  # 6039명유저중마지막추가된유저 6039번이며,총36040개) 
print(movies_to_idx[my_favorite[0]]) 

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해봄 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movies_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱 
temp_movie_data = ratings['movie_id'].map(movies_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

## timestamp 제외: my favotite 5개 추가로 rufcmrcl 5개 발생된데다가,학습에 필요없어서 제외 
ratings = ratings.iloc[:,:-1]
print("ratings", ratings.shape, ratings.head(),ratings.tail())


Legends of the Fall
mov_id 266 mov_real_title Legends of the Fall (1994)
Jurassic Park
mov_id 1544 mov_real_title Lost World: Jurassic Park, The (1997)
Braveheart
mov_id 110 mov_real_title Braveheart (1995)
Terminator
mov_id 589 mov_real_title Terminator 2: Judgment Day (1991)
Dances with Wolves
mov_id 590 mov_real_title Dances with Wolves (1990)
my_favorite_movie_id 확인:  [266, 1544, 110, 589, 590]
my_favorite_movie_real_title 확인:  ['Legends of the Fall (1994)', 'Lost World: Jurassic Park, The (1997)', 'Braveheart (1995)', 'Terminator 2: Judgment Day (1991)', 'Dances with Wolves (1990)']
ratings.tail(5)    user_id  movie_id  counts  timestamp
0  bluesky       266       5        NaN
1  bluesky      1544       5        NaN
2  bluesky       110       5        NaN
3  bluesky       589       5        NaN
4  bluesky       590       5        NaN
merge.tail(5)    user_id  movie_id  counts  timestamp title genre
0  bluesky       266     5.0        NaN   NaN   NaN
1  bluesky      1544     5.0   

##### [내가 선호하는 영화 5개추가] my_favorite_title = ['Legends of the Fall' , 'Jurassic Park' ,'Braveheart' ,'Terminator' ,'Dances with Wolves']

#### 4) CSR matrix를 직접 만들어 봅시다.

In [10]:
#[[ your code ]]

from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movies = ratings['movie_id'].nunique()
print("num_user,num_movies", num_user,num_movies)

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movies))
print("csr_data", csr_data.shape, csr_data)  #[:10],csr_data[-10:] )
print("num_user,num_movies", num_user,num_movies)

num_user,num_movies 6040 3628
csr_data (6040, 3628)   (0, 0)	5
  (0, 1)	3
  (0, 2)	3
  (0, 3)	4
  (0, 4)	5
  (0, 5)	3
  (0, 6)	5
  (0, 7)	5
  (0, 8)	4
  (0, 9)	4
  (0, 10)	5
  (0, 11)	4
  (0, 12)	4
  (0, 13)	4
  (0, 14)	5
  (0, 15)	4
  (0, 16)	3
  (0, 17)	4
  (0, 18)	5
  (0, 19)	4
  (0, 20)	3
  (0, 21)	3
  (0, 22)	5
  (0, 23)	5
  (0, 24)	3
  :	:
  (6038, 2311)	4
  (6038, 2317)	5
  (6038, 2386)	4
  (6038, 2394)	5
  (6038, 2424)	4
  (6038, 2437)	4
  (6038, 2446)	5
  (6038, 2471)	4
  (6038, 2511)	5
  (6038, 2523)	4
  (6038, 2559)	3
  (6038, 2560)	4
  (6038, 2631)	5
  (6038, 2648)	4
  (6038, 2654)	5
  (6038, 2738)	4
  (6038, 2740)	5
  (6038, 2857)	5
  (6038, 2860)	3
  (6038, 3311)	5
  (6039, 82)	5
  (6039, 87)	5
  (6039, 92)	5
  (6039, 116)	5
  (6039, 356)	5
num_user,num_movies 6040 3628


#### [CSR 매트릭스에대한 관찰] CSR 매트릭스는 출력된 결과를 봤을때, unique한 users와 unique한 movie의 개수를 row,column으로 구성하되, 일반적인 매트릭스처럼 가로,세로 shape대로, matrix를 펼쳐놓지 않고, row,column역할은 그 shape에 들어가는 좌표상의 요소의 위치를 나타내는 pointer 역할을 하고,요소인 별점값은 별도 메모리에 저장해서, als모델에서, query가 있을때, 해당 pointer를 사용하여 값을 제공하는  데이터구조를 이용해서, 메모리를 효율적으로 쓴다는 것을 알게되었슴

#### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [11]:
#[[ your code ]]

from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

##  Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=25, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose)
csr_data_transpose = csr_data.T
print("csr_data_transpose", csr_data_transpose.shape, csr_data_transpose)

## 모델 훈련
als_model.fit(csr_data_transpose)


csr_data_transpose (3628, 6040)   (0, 0)	5
  (1, 0)	3
  (2, 0)	3
  (3, 0)	4
  (4, 0)	5
  (5, 0)	3
  (6, 0)	5
  (7, 0)	5
  (8, 0)	4
  (9, 0)	4
  (10, 0)	5
  (11, 0)	4
  (12, 0)	4
  (13, 0)	4
  (14, 0)	5
  (15, 0)	4
  (16, 0)	3
  (17, 0)	4
  (18, 0)	5
  (19, 0)	4
  (20, 0)	3
  (21, 0)	3
  (22, 0)	5
  (23, 0)	5
  (24, 0)	3
  :	:
  (2311, 6038)	4
  (2317, 6038)	5
  (2386, 6038)	4
  (2394, 6038)	5
  (2424, 6038)	4
  (2437, 6038)	4
  (2446, 6038)	5
  (2471, 6038)	4
  (2511, 6038)	5
  (2523, 6038)	4
  (2559, 6038)	3
  (2560, 6038)	4
  (2631, 6038)	5
  (2648, 6038)	4
  (2654, 6038)	5
  (2738, 6038)	4
  (2740, 6038)	5
  (2857, 6038)	5
  (2860, 6038)	3
  (3311, 6038)	5
  (82, 6039)	5
  (87, 6039)	5
  (92, 6039)	5
  (116, 6039)	5
  (356, 6039)	5


  0%|          | 0/25 [00:00<?, ?it/s]

##### 모델학습은 iteration 25로 했슴 (LMS교재실습시 했던 15로 했을때보다, 소폭의 개선이 있었슴)

#### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [12]:
#[[ your code ]]

## 내가 선호하는 5가지 영화 중 하나
bluesky, my_favorite0 = user_to_idx['bluesky'], movies_to_idx[my_favorite[0]]
bluesky_vector, my_favorite0_vector = als_model.user_factors[bluesky], als_model.item_factors[my_favorite0]
#print("bluesky_vector",bluesky_vector, "my_favorite0_vector",my_favorite0_vector)

# bluesky, my_favorite0를 내적하는 코드
print("내가 선호하는 5가지 영화 중 하나의 내적벡터: ", np.dot(bluesky_vector, my_favorite0_vector))

## 그외의 영화
SilenceoftheLambs = movies_to_idx[593]   # Silence of the Lambs, The (1991) 
SilenceoftheLambs_vector = als_model.item_factors[SilenceoftheLambs]
print("그외의 영화의 내적벡터: ", np.dot(bluesky_vector, SilenceoftheLambs_vector))


내가 선호하는 5가지 영화 중 하나의 내적벡터:  0.5127211
그외의 영화의 내적벡터:  0.12351194


##### [평가] 내가 선호하는 영화의 내적벡터는 0.512로 0.5를 넘겼고, 그외의 영화는 0.12이었는데,  추천시스템이 어느정도 효과를 발휘하는 것 같음

#### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [22]:
#[[ your code ]]
favorite_movie = my_favorite[0]
mov_id = movies_to_idx[favorite_movie]
similar_movie = als_model.similar_items(mov_id, N=10)
print("similar_movie", similar_movie)

#movies_to_idx 를 뒤집어, index로부터 merge파일에 있는 Original movie_id 을 얻는 dict를 생성 
idx_to_movies_original = {v:k for k,v in movies_to_idx.items()}
mov_id1 = [idx_to_movies_original[i[0]] for i in similar_movie]  # merge파일에 있는 Original movie_id
print("merge파일에 있는 Original movie_id == mov_id1", mov_id1)

# Original movie_id를 통해서, merge파일에 있는 정확한 title(영화제목)을 얻음
my_similar_mov =[] 
for mov in mov_id1:
    mov_real_title1 = merge.loc[merge['movie_id']==mov, 'title'].values[0] 
    my_similar_mov.append(mov_real_title1)
print("내가 좋아하는 영화와 비슷한 영화 추천 10선: ",my_similar_mov)


similar_movie [(356, 1.0), (337, 0.44827062), (3466, 0.43201005), (105, 0.43059483), (409, 0.41323242), (1509, 0.40459737), (181, 0.395986), (2652, 0.39441276), (835, 0.39405838), (1474, 0.39394915)]
merge파일에 있는 Original movie_id == mov_id1 [266, 383, 989, 1408, 524, 1183, 1379, 458, 553, 144]
내가 좋아하는 영화와 비슷한 영화 추천 10선:  ['Legends of the Fall (1994)', 'Wyatt Earp (1994)', 'Schlafes Bruder (Brother of Sleep) (1995)', 'Last of the Mohicans, The (1992)', 'Rudy (1993)', 'English Patient, The (1996)', 'Young Guns II (1990)', 'Geronimo: An American Legend (1993)', 'Tombstone (1993)', 'Brothers McMullen, The (1995)']


##### [평가] 않보아서, 모르는 영화도 있어, 그것들까지 보아야 제대로 평가가되겠지만, 본 영화들기준으로 보면 대체로 다 좋아하는 영화들인 것으로 보아,추천시스템이 어느 정도이상의 성능이 있는 것으로 보임 

#### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [54]:
#[[ your code ]]
user = user_to_idx['bluesky']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=10, filter_already_liked_items=True)
print("신규 id기준의 movie_recommended",movie_recommended)

# merge 파일의 title 컬럼에있는 Original 영화_id를 가져옴 : 위에서 만든 idx_to_movies_original dict이용
movie_recommended1 = [[idx_to_movies_original[i[0]],i[1]] for i in movie_recommended]
print("merge파일에 있는 Original movie_id기준의 movie_recommended1:", movie_recommended1)

# Original movie_id를 통해서, merge파일에 있는 정확한 title(영화제목)을 얻음 
my_best_recommended_mov =[] 
for mov in movie_recommended1:
    mov_real_title2 = merge.loc[merge['movie_id']==mov[0], 'title'].values[0] 
    mov_preffer_score = mov[1]
    my_best_recommended_mov.append((mov_real_title2,mov_preffer_score))
print("내가 가장 좋아할 만한 영화들을 추천 10선: ", my_best_recommended_mov)


신규 id기준의 movie_recommended [(107, 0.6515017), (175, 0.45561835), (48, 0.37551433), (80, 0.33206075), (124, 0.31690586), (62, 0.31331566), (346, 0.2859893), (250, 0.2752167), (14, 0.25653416), (141, 0.25455788)]
merge파일에 있는 Original movie_id기준의 movie_recommended1: [[480, 0.6515017], [1580, 0.45561835], [2028, 0.37551433], [1259, 0.33206075], [2571, 0.31690586], [2916, 0.31331566], [3753, 0.2859893], [2291, 0.2752167], [1035, 0.25653416], [457, 0.25455788]]
내가 가장 좋아할 만한 영화들을 추천 10선:  [('Jurassic Park (1993)', 0.6515017), ('Men in Black (1997)', 0.45561835), ('Saving Private Ryan (1998)', 0.37551433), ('Stand by Me (1986)', 0.33206075), ('Matrix, The (1999)', 0.31690586), ('Total Recall (1990)', 0.31331566), ('Patriot, The (2000)', 0.2859893), ('Edward Scissorhands (1990)', 0.2752167), ('Sound of Music, The (1965)', 0.25653416), ('Fugitive, The (1993)', 0.25455788)]


##### [평가] als_model.recommend를 통해 추천받은 10편의 영화들중 7편은 보았고,3편은 않본 영화들이지만, 본 것기준으로는 7편중 6편이 좋아하는 영화임,  본것기준으로 평가하면  85.7%( = (3/7)*100) 정도의 정확도 갖는듯


#### 9) MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도를 측정하고 의미를 분석해보았다.

###### 가) 유저 선호도

내가 가장 좋아할 만한 영화들을 추천 10선은 

1.('Jurassic Park (1993)', 0.6515017), 

2.('Men in Black (1997)', 0.45561835),

3.('Saving Private Ryan (1998)', 0.37551433),

4.('Stand by Me (1986)', 0.33206075),

5.('Matrix, The (1999)', 0.31690586),

6.('Total Recall (1990)', 0.31331566), 

7.('Patriot, The (2000)', 0.2859893),

8.('Edward Scissorhands (1990)', 0.2752167),

9.('Sound of Music, The (1965)', 0.25653416), 

10.('Fugitive, The (1993)', 0.25455788)]

위의 추천영화들을 보면,Jurassic Park (1993)이 MF모델의 선호도예측점수 0.6515로 1위이고,
2위가 Men in Black (1997)으로 0.4556이며, 10위가 Fugitive, The (1993)로서, 0.25455788 임

선호도 예측점수가 0.5를 초과하거나 근접한 영화는 1,2위인 Jurassic Park (1993)과 Men in Black (1997) 두편뿐이지만, 5위, 6위, 7위, 8위도 내가 선호하는 영화임,

###### 이로 미루어 볼때 선호도가 꼭 0.5 이상이 좋을 것이란 기준은 주관적인 판단으로 각자마다 다를수 있으니, 선호도 예측점수만으로 MF모델의 성능을 평가하기 보다는, 상위 10위이내에서, 좋은 평점을 줄수있는 영화가 몇개인가로 MF모델을 평가하는 것도 좋을듯함

###### 나의 경우는 10편중 7편만 보았기 때문에, 3편을 더 보고 평가해야하지만, 본 것만 가지고 평가한다면,85.7%( = (3/7)*100) 정도의 정확도 갖는 것으로 평가됨



###### 나) 아이템간 유사도: 

In [82]:
for idx in range(len(movie_recommended)):
    item_id1 = movie_recommended[idx][0] 
    #print(item_id1,movie_recommended)

    # 추천종목별 유사도상위 파악: als_model.similar_items()
    similar_movie1 = als_model.similar_items(item_id1, N= 10)
    #print(similar_movie1)
    
    # merge 파일의 title 컬럼에있는 Original 영화_id를 가져옴 : 위에서 만든 idx_to_movies_original dict이용
    item_id1_title = merge.loc[merge['movie_id']==idx_to_movies_original[item_id1], 'title'].values[0]
    #print("merge파일에 있는 Original movie_id기준의 item_id1_title:", item_id1_title)
    
    similar_movie1 = [[idx_to_movies_original[i[0]],i[1]] for i in similar_movie1]
    #print("merge파일에 있는 Original movie_id기준의 similar_movie1:", similar_movie1)
        
    my_similarity_mov =[] 
    for mov in similar_movie1:
        mov_real_title2 = merge.loc[merge['movie_id']==mov[0], 'title'].values[0] 
        mov_preffer_score = mov[1]
        my_similarity_mov.append((mov_real_title2,mov_preffer_score))
    print("idx",idx,"\n", f"추천 Best 10선중 { item_id1_title}과 유사성갖은 영화 상위 10개의 유사도: ",my_similarity_mov)
    print('-----'*10)

idx 0 
 추천 Best 10선중 Jurassic Park (1993)과 유사성갖은 영화 상위 10개의 유사도:  [('Jurassic Park (1993)', 0.99999994), ('Men in Black (1997)', 0.8428679), ('Terminator 2: Judgment Day (1991)', 0.72623765), ('Total Recall (1990)', 0.5656872), ('Matrix, The (1999)', 0.5477858), ('Braveheart (1995)', 0.47209877), ('Lost World: Jurassic Park, The (1997)', 0.4300171), ('Schlafes Bruder (Brother of Sleep) (1995)', 0.42711142), ('Independence Day (ID4) (1996)', 0.3948178), ('Fifth Element, The (1997)', 0.36734465)]
--------------------------------------------------
idx 1 
 추천 Best 10선중 Men in Black (1997)과 유사성갖은 영화 상위 10개의 유사도:  [('Men in Black (1997)', 0.99999976), ('Jurassic Park (1993)', 0.8428679), ('Terminator 2: Judgment Day (1991)', 0.61973083), ('Total Recall (1990)', 0.5726202), ('Matrix, The (1999)', 0.48298433), ('Independence Day (ID4) (1996)', 0.47365543), ('Fifth Element, The (1997)', 0.4495876), ('Lost World: Jurassic Park, The (1997)', 0.4200113), ('Galaxy Quest (1999)', 0.41002697), ('Face

아이템간 유사도는 10편 모두 for loop 코드를 사용해서,
개별 영화별로 similarity가 높은 영화들의 유사도를 살펴보았는데,

###### 순위 1위 영화인  Jurassic Park은 같은 SF영화인 Men in Black과 0.8428679의 유사도를, Terminator 2는 0.72623765의 유사도, Total Recall 0.5656872, Matrix 0.5477858 공상과학영화와의 유사도가 높은 것으로 나타남

###### 순위2위~순위 10위까지를 하나 하나 살펴보면, 영화의 유사도가 영화의 장르와 주인공의 휴머니즘과 정신적인 용기, 등 인간적인 매력이란 요소를 기준으로, 유사도를 측정한 것같으며, 복잡하지 않고, 벡터화와 행렬화를 통한 특이치 분석속에서 그러한 특징을 뽑아낼수 있고, 작은 횟수의 학습만으로도, 유저의 특성과 아이템의 특성과 유사도를 상당히 잘평가할수 있다는 점이 신기하고도,매력적이었슴


###### 다) 추천 영화에대한 기여도: MF모델이 예측한 선호도 1위인 'Jurassic Park (1993)'에대한 기여도분석임


In [37]:
# Explain
print(merge.loc[merge['movie_id']==movie_recommended1[0], 'title'].values[0] )
explain = als_model.explain(user, csr_data, itemid=movie_recommended[0][0])
print("explain", explain)

# merge 파일의 title 컬럼에있는 Original 영화_id를 가져옴 
contriv = [(idx_to_movies_original[i[0]], i[1]) for i in explain[1]]
print("[Original movie_id기준]'Jurassic Park (1993)'추천에 기여한 영화들의 기여도 contriv:",contriv)

# Original movie_id를 통해서, merge파일에 있는 정확한 title(영화제목)을 얻음 
contriv_list =[]
for movid in contriv:
    contriv_list.append((merge.loc[merge['movie_id']==movid[0],'title'].values[0],movid[1]))
    
print("[영화제목기준]MF모델의 선호도예측점수 1위'Jurassic Park (1993)'의 선호도값 :", explain[0])  

print("[영화제목기준]'Jurassic Park (1993)'추천에 기여한 영화들의 기여도 contriv_list:",contriv_list)    


Jurassic Park (1993)
explain (0.6432893038537915, [(92, 0.26747505023292983), (87, 0.1312191649605606), (82, 0.11625730045535013), (116, 0.06595248846263349), (356, 0.0623852997423174)], (array([[ 6.89116531e-01,  8.11471635e-02,  8.65960679e-02, ...,
         1.36733201e-01,  6.09535421e-02,  1.19122715e-01],
       [ 5.59198519e-02,  6.92142979e-01,  1.43868261e-01, ...,
         1.01161373e-01,  1.20939035e-01,  1.05890921e-01],
       [ 5.96747820e-02,  1.06604432e-01,  6.43294761e-01, ...,
         6.49922734e-02,  7.74907860e-02,  7.40052743e-02],
       ...,
       [ 9.42251093e-02,  8.11136454e-02,  6.82036574e-02, ...,
         6.09910845e-01, -3.49813576e-04,  4.20724177e-03],
       [ 4.20040935e-02,  8.86533112e-02,  7.25270425e-02, ...,
         6.65908275e-02,  6.20234077e-01,  3.43583972e-02],
       [ 8.20894321e-02,  8.29581282e-02,  7.31571067e-02, ...,
         7.12985834e-02,  1.03897168e-01,  6.09944378e-01]]), False))
[Original movie_id기준]'Jurassic Park (1993)'추천에

위의 als_model.explain 코드 실행결과를 보면,

###### Jurassic Park (1993)의 전체 선호도 = 0.6432893038537915

'Jurassic Park (1993)'추천에 기여한 영화들의 기여도 = [('Terminator 2: Judgment Day (1991)', 0.26747505023292983), ('Braveheart (1995)', 0.1312191649605606), ('Lost World: Jurassic Park, The (1997)', 0.11625730045535013), ('Dances with Wolves (1990)', 0.06595248846263349), ('Legends of the Fall (1994)', 0.0623852997423174)]

로서, 

###### Terminator 2가 0.2674의 기여도(전체 선호도의 41.6%기여),
###### Braveheart가 0.1312(전체 선호도의 20.4%기여),
###### Lost World: Jurassic Park가 0.1162(전체 선호도의 18.1%기여),
###### Dances with Wolves가 0.065(전체 선호도의 10.0%기여),
###### Legends of the Fall이 0.062(전체 선호도의 9.7%기여) 의 기여도를 갖는 것으로 보아,

###### MF모델이 나는 <SF와 휴머니즘이 결합된 영화>에 선호도를 갖고 있다고 학습하고서, 예측한 것으로 보이며,이는 기본이상의 상당히 양호한 예측인 것으로 보임


## 회고

본 프로젝트는 추천시스템에 관한 것으로서, MF 모델 학습 방법을 토대로, 각자 개인별로 좋아할 만한 영화 추천 시스템을 제작해 보고, 모델이 주관적인 취향을 잘파악해서, 마음에드는 영화를 잘추천했는지, 모델의 성능을
평가해보는 프로젝트입니다.

사용한 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터를 사용하였습니다. 데이터 총수는 1000209개였는데,
별점 3점이상만 선택하여 총 836478개로 축소하였습니다.

데이터를 분석해보면, ratings에 있는 유니크한 영화 개수는 3628개(별점3점이상 전체데이터수 836478개중),
ratings에 있는 유니크한 사용자 수 6039개(별점3점이상 전체데이터수 836478개중),가장 인기 있는 영화 30개(인기순)는 총30개중 상위 5개만 예시해보면, American Beauty (1999), Star Wars: Episode IV - A New Hope (1977),
Star Wars: Episode V - The Empire Strikes Back (1980), Star Wars: Episode VI - Return of the Jedi (1983),Jurassic Park (1993)이었습니다.
메타 데이터 movies의 영화제목 총수는 3883개였습니다. 

그리고, 앞으로 만들 추천시스템에서 나의 영화선호도를 파악하게 하기위해서, 내가 선호하는 영화 5개를 입력데이터로 쓰일 ratings 파일에 추가하였습니다.

my_favorite_title = ['Legends of the Fall' , 'Jurassic Park' ,'Braveheart' ,'Terminator' ,'Dances with Wolves'의 5개 입니다.¶

ratings에 내가 선호하는 영화를 추가한후, 신규로 생긴 user id와 movie id를 기존의 user와 영화를 포함하여 전체를 통합하여, 새로이 indexing하고서, 학습에 필요없는
timestamp 컬럼을 제거하고서, user_id, movie_id, counts 컬럼등 숫자로만 구성된 벡터 데이터를 완성하였습니다.

이때 새로이 indexing이 되다 보니,영화id가 enumerate()에의해 변경되게 도기때문에,메터데이터인 movies를 ratings와 병합한 merge파일을 별도로 만들어 끝까지 유지하며, idx_to_movies_original과 merge파일의 title컬럼을 통해서,새로운 영화인덱스가 정확한 영화제목과 정확히 매칭이 되도록 하였습니다.
                     
이 데이터를 메모리효율적, 학습효율적으로 만들기위해, csr matrix로 만들었는데, 
LMS교재에서 csr의 원리를 알게해주기위해 실습했던 csr_matrix((ratings.counts,indices, indptr; shape= (num_user, num_movies))방식이 아닌,
csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movies))방식으로 만들었습니다.
                        
CSR Matrix는 Sparse한 matrix에서 0 이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여,
메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조라고 교재에서 배웠는데,실제로 만들어서, 출력해보니, 교재에서 설명한 개념이 이해가 되었습니다.
                                         
출력된 결과를 봤을때, unique한 users와 unique한 movie의 개수를 row,column으로 shape를 구성하되,
일반적인 매트릭스처럼 가로,세로 shape대로, matrix를 펼쳐놓지 않고, 
row,column역할은 그 shape에 들어가는 좌표상의 요소의 위치를 나타내는 pointer및 좌표역할을 하고,
요소인 별점값은 별도 메모리에 저장해서, als모델에서, query가 있을때, 해당 pointer를 사용하여 값을 제공하는  데이터구조를 이용해서,
메모리공간을 대폭 절감해여, 효율적으로 쓴다는 것을 알게되었습니다.
                                         
학습시킬 모델은 Matrix Factorization 모델로서,
암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지인 implicit 패키지의
als(AlternatingLeastSquares)를 사용하였는데, 그 이유는 Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은
잘 수렴하지 않기 때문에,한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 방식이기 때문이었습니다.
                                 
모델학습은 iteration 25, factors=100, regularization=0.01 으로 했습니다(나머지는 LMS교재실습시 했던 대로 했고,iteration만 15에서 25로 늘려보았는데,더 많이 늘리지 않은 이유는 과대적합을 검증할 장치가 없어서 였는데, 성능이 0.49 에서 0.51로 소폭의 개선이 있었습니다.    
 
6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.에서는
내가 선호하는 영화의 내적벡터는 0.512로 0.5를 넘겼고, 그외의 영화는 0.12이었는데, 추천시스템이 어느정도 효과를 발휘하는 것 같습니다.

7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.에서는 10편을 추천받았는데, 않보아서, 모르는 영화가 6편 있어, 그것들까지 보아야 제대로 평가가되겠지만, 본 영화들 4편 기준으로 보면 4편 다 좋아하는 영화들인 것으로 보아, 추천시스템이 어느 정도이상의 성능이 있는 것으로 보입니다. 4편으로 평가하다면,Map라고 하긴 수량이 적지만 4편중에 3편이 좋아하는 영화라서, 75%의 성능을 보입니다.

8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다에서는 als_model.recommend를 통해 추천받은 10편의 영화들은 나의 경우는 10편중 7편만 보았기 때문에, 3편을 더 보고 평가해야하지만, 
본 것만 가지고 평가한다면,85.7%(=(3/7)*100)정도의 정확도를 갖는 것으로 평가되었습니다.

#### 9) MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도를 측정하고 의미를 분석해보았다는
가) 유저 선호도

내가 가장 좋아하는 영화로 추천된들을 보면,Jurassic Park (1993)이 MF모델의 선호도예측점수 0.6515로 1위이고, 2위가 Men in Black (1997)으로 0.4556이며, 10위가 Fugitive, The (1993)로서, 0.25455788 입니다.

선호도 예측점수가 0.5를 초과하거나 근접한 영화는 1,2위인 Jurassic Park (1993)과 Men in Black (1997) 두편뿐이지만, 5위, 6위, 7위, 8위도 내가 선호하는 영화입니다.

이로 미루어 볼때 선호도가 꼭 0.5 이상이 좋을 것이란 기준은 주관적인 판단으로 각자마다 다를수 있으니, 선호도 예측점수만으로 MF모델의 성능을 평가하기 보다는, 상위 10위이내에서, 좋은 평점을 줄수있는 영화가 몇개인가로 MF모델을 평가하는 것도 좋을듯하고, 나의 경우는 10편중 7편만 보았기 때문에, 3편을 더 보고 평가해야하지만, 본 것만 가지고 평가한다면,85.7%( = (3/7)*100) 정도의 정확도 갖는 것으로 평가되니다.

나) 아이템간 유사도

아이템간 유사도는 10편 모두 for loop 코드를 사용해서, 개별 영화별로 similarity가 높은 영화들의 유사도를 살펴보았는데,순위 1위 영화인 Jurassic Park은 같은 SF영화인 Men in Black과 0.8428679의 유사도를, Terminator 2는 0.72623765의 유사도, Total Recall 0.5656872, Matrix 0.5477858 공상과학영화와의 유사도가 높은 것으로 나타났습니다. 순위2위~순위 10위까지를 하나 하나 살펴보면, 영화의 유사도가 영화의 장르와 주인공의 휴머니즘과 정신적인 용기, 등 인간적인 매력이란 요소를 기준으로, 유사도를 측정한 것같으며, 복잡하지 않고, 벡터화와 행렬화를 통한 특이치 분석속에서 그러한 특징을 뽑아낼수 있고, 작은 횟수의 학습만으로도, 유저의 특성과 아이템의 특성과 유사도를 상당히 잘평가할수 있다는 점이 신기하고도,매력적이었습니다.

다) 추천 영화에대한 기여도: 

시간관계상 10개 전체를 하기보다, MF모델이 예측한 선호도 1위인 'Jurassic Park (1993)'에대한 기여도분석을 했습니다.

Jurassic Park (1993)의 전체 선호도 = 0.6432893038537915
'
Terminator 2가 0.2674의 기여도(전체 선호도의 41.6%기여),

Braveheart가 0.1312(전체 선호도의 20.4%기여),

Lost World: Jurassic Park가 0.1162(전체 선호도의 18.1%기여),

Dances with Wolves가 0.065(전체 선호도의 10.0%기여),

Legends of the Fall이 0.062(전체 선호도의 9.7%기여) 의 기여도를 갖는 것으로 보아,
MF모델이 나는 <SF와 휴머니즘이 결합된 영화>에 선호도를 갖고 있다고 학습하고서, 예측한 것으로 보이며,이는 기본이상의 상당히 양호한 예측인 것으로 보임
                     
결론적으로 MF 방식의 추천시스템은 효과적인 성능을 보이는 것으로 파악되었습니다. 재미있고 좋은 프로젝트 감사합니다.                     


루브릭

아래의 기준을 바탕으로 프로젝트를 평가합니다.

평가문항	
상세기준

1. CSR matrix가 정상적으로 만들어졌다.

사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.

2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.

사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.

3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.

MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도를 측정하고 의미를 분석해보았다.